# WIP: Conten Based filtering using audio features, album_uri and artist_uri

In [1]:
import import_ipynb

In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import scipy.sparse as sps

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from category_encoders import TargetEncoder
from time import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#own functions
from evaluation import DCG
from evaluation import nDCG
from evaluation import R_Precision


%matplotlib inline

importing Jupyter notebook from evaluation.ipynb
DCG = 0.5
IDCG = 1.0
nDCG = 0.5


# Define function to look up similar items 

In [3]:
with open('../data-processed/full-data/track_descriptions.json') as json_file:
    D_desc = json.load(json_file)
    
D_desc['spotify:track:0UaMYEvWZi0ZqiDOoHU3YI']

['Lose Control (feat. Ciara & Fat Man Scoop)', 'Missy Elliott', 'The Cookbook']

In [4]:
#---------------------
# FIND SIMILAR ITEMS
#---------------------

def similar_items(track_uri, top_n, similarity_matrix):
    """
    input: 
        track_uri
    output: 
        top_n recommended track_uris
    """
    #print('seed track is not included')
    D_rec={}
    #time0 = time()
    idx = D_track_uri_to_id[track_uri]
    similarity_array = cosine_similarity(similarity_matrix[idx:idx+1,:], similarity_matrix)
    recc_tracks = np.fliplr(similarity_array.argsort())
    for el in recc_tracks[0][1:top_n+1]:
        D_rec[D_track_id_to_uri[el]]=similarity_array[0][el]
    #print(f'{time()-time0:0.2f}sec')
    return D_rec

In [5]:
#-------------------------------------
# FIND SIMILAR ITEMS WITH DESCRIPTION
#-------------------------------------

def similar_items_with_description(track_uri, top_n, similarity_matrix):
    """
    input: 
        track_uri
    output: 
        top_n recommended track_uris with description as dictionary
    """
    print('CB audio features - first tracks returned is the seed track')
    D_rec={}
    time0 = time()
    idx = D_track_uri_to_id[track_uri]
    similarity_array = cosine_similarity(similarity_matrix[idx:idx+1,:], similarity_matrix)
    recc_tracks = np.fliplr(similarity_array.argsort())
    for el in recc_tracks[0][0:top_n+1]:
        D_rec[D_track_id_to_uri[el]]=D_desc[D_track_id_to_uri[el]]
    print(f'{time()-time0:0.2f}sec')
    return list(D_rec.values())

In [6]:
def similar_items_with_description_external(track_uri, top_n):
    print('CB audio features - first tracks returned is the seed track')
    D_rec={}
    time0 = time()
    idx = D_track_uri_to_id[track_uri]
    similarity_array = cosine_similarity(X_transformed[idx:idx+1,:], X_transformed)
    recc_tracks = np.fliplr(similarity_array.argsort())
    for el in recc_tracks[0][0:top_n+1]:
        D_rec[D_track_id_to_uri[el]]=D_desc[D_track_id_to_uri[el]]
    print(f'{time()-time0:0.2f}sec')
    return list(D_rec.values())

In [7]:
import operator

#----------------------------------------------------------------------
# PREDICT TOP_N TRACKS FOR A PID AND EVALUATE AGAINST THE GROUND TRUTH
#----------------------------------------------------------------------

def predict_and_evaluate_top_n(pid,top_n):
    """
    return
        top_n predicted track_ids
        ground_truth : track_ids in the hold_out
        R_Prec
        NDCG
    """
    train_array_track_ids = track_id_array[M[pid].toarray()[0].astype(bool)]
    
    D_pred={}
    
    topn_n_by_track = int(top_n/len(train_array_track_ids))*2
    
    for el in train_array_track_ids:
        D_pred.update(similar_items(D_track_id_to_uri[el],topn_n_by_track,X_transformed))
    
    D_pred = dict(sorted(D_pred.items(), key=operator.itemgetter(1),reverse=True))
    ground_truth = ev_set_arr[ev_set_arr[:,0]==pid][:,2]
    L_pred = list(D_pred.keys())[:top_n]
    R_Prec = R_Precision(L_pred[:len(ground_truth)],ground_truth)
    res = [int(el in list(ground_truth)) for el in L_pred]
    NDCG = nDCG(res)[1]
    return L_pred, ground_truth, R_Prec, NDCG, res

# Data transformation, PCA and merging 

In [8]:
df_merged = pd.read_csv('../data-processed/transformation-matrices/cb_df_merged.csv')

MemoryError: Unable to allocate 14.0 GiB for an array with shape (833, 2262190) and data type float64

In [ ]:
df_merged.shape

In [ ]:
df_merged.head()

In [ ]:
#df_merged = df_merged.fillna(0)

In [ ]:
#df_merged.to_csv('../data-processed/transformation-matrices/cb_df_merged.csv', index = None)

In [ ]:
X_transformed = np.asarray(df_merged)

In [ ]:
X_transformed.shape

# Find similar tracks

<img src="tenor.gif">

In [ ]:
similar_items_with_description('spotify:track:3ZOEytgrvLwQaqXreDs2Jx',10,X_transformed)